# Inferencia de las fuerzas moleculares de los elementos del dataset QM9

## Importación de módulos

In [ ]:
import torch
import numpy as np
import pandas as pd
import os
from scripts.qm9 import QM9

## Definición de directorios

In [ ]:
path_splits_npy = "/workspace/clase_torchmd-net/implementacion/output/splits.npz"

# Ruta al dataset que uso torchmd-net
path_torchmdnet_dataset = "/workspace/clase_torchmd-net/data/dataset_qm9.pt"

# Ruta al modelo entrenado
path_trained_model="/workspace/clase_torchmd-net/implementacion/output/epoch=669-val_loss=0.0003-test_loss=0.0064.ckpt"

In [ ]:
# Cargar archivo del dataset que utiliza torchmd-net
torchmdnet_dataset = torch.load(path_torchmdnet_dataset)

# Cargar archivo splits
splits_data = np.load(path_splits_npy)

In [ ]:
# Acceder a la matriz específica dentro del archivo npz
splits = splits_data['idx_test'] 

# Indexar el dataset con los índices de splits
dataset_torchmdnet_test = torchmdnet_dataset[splits]

In [ ]:
from torchmdnet.models.model import load_model
model= load_model(path_trained_model, derivative=True)

In [ ]:
if not os.path.exists('data/inferencias_test.dat'):
    with open('data/inferencias_test.dat', 'w') as file:
        for molecula in dataset_torchmdnet_test:
            z_sample = molecula.z
            pos_sample = molecula.pos
            n_energy_sample = molecula.y
            # Hacer la inferencia
            n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
            # Transformar la inferencia
            n_energy_inferred = float(n_energy_inferred)
            n_energy_sample = float(n_energy_sample)        
            # Guardar los datos en el archivo
            file.write(f"{n_energy_sample} {n_energy_inferred}\n")
else:
    print("El archivo 'inferencias_test.dat' ya existe. No se ha realizado ninguna acción.")